# 파이토치 RNN 분류 모델
- 코드 출처: https://tutorials.pytorch.kr/intermediate/char_rnn_classification_tutorial.html

이름: Ślusàrski

## 신경망 생성

- 모델 생성 방법 참고 : https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html

## 모델 학습
- 준비

## 손실함수

## 모델 학습

## 모델 테스트

## 사용자 입력으로부터의 실행